# Phishing Domain Detection
---

BONIFASIUS TARIGAN - 1301213235

# Installs
---

In [ ]:


!pip install -q autoviz
!pip install -q -U --pre pycaret

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.5/67.5 kB 936.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━ 168.6/255.9 MB 29.5 MB/s eta 0:00:03

In [ ]:
!pip -q install transformers
!pip -q install torch

# Imports
---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [ ]:
!pip install --upgrade autoviz
!pip show autoviz

In [ ]:
# from autoviz.classify_method import data_cleaning_suggestions ,data_suggestions
from autoviz.AutoViz_Class import data_cleaning_suggestions
from pycaret import regression
from sklearn.model_selection import cross_val_score

# Data Loading
---

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)
# !ls

In [ ]:
# import os
# os.chdir("/content/gdrive/My Drive/KULIAH/SKC/Dataset")
# !ls


In [ ]:
# df = pd.read_csv('/kaggle/input/malicious-urls-dataset/malicious_phish.csv', nrows = 1000)
# df = pd.read_csv('/content/gdrive/My Drive/KULIAH/SKC/Dataset/malicious_phish.csv')
df = pd.read_csv('malicious_phish.csv')


In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

# EDA
---

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
data_cleaning_suggestions(df)

# 1. Handling Categorical Values

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# get list of categorical columns
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
cat_cols

In [ ]:
# Initialize the LabelEncoder
le = LabelEncoder()

# Fit and transform the data
encoded_data = le.fit_transform(df['type'])

print(encoded_data)

In [ ]:
df

In [ ]:
df['type'] = encoded_data

In [ ]:
df

# 2. Handling Null Values

In [ ]:
df.isnull().any()

In [ ]:
df.dtypes

# 3. Class Distributions

In [ ]:
# Count the number of instances in each class
class_counts = df['type'].value_counts()

# Print the class distribution
print('Class distribution:')
print(class_counts)

In [ ]:
sns.countplot(x = 'type', data = df, order = df['type'].value_counts().index)

# Feature Extraction using Bert Model
---

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

# Load the pre-trained BERT model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# # Define a function to extract features for each transaction
# def extract_features(text):
#     # Tokenize the text
#     input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
#     # Get the hidden states for each token
#     with torch.no_grad():
#         outputs = model(input_ids)
#         hidden_states = outputs[2]
#     # Concatenate the last 4 hidden states
#     token_vecs = []
#     for layer in range(-4, 0):
#         token_vecs.append(hidden_states[layer][0])
#     # Calculate the mean of the last 4 hidden states
#     features = []
#     for token in token_vecs:
#         features.append(torch.mean(token, dim=0))
#     # Return the features as a tensor
#     return torch.stack(features)

In [ ]:
# # Extract features for each transaction
# features = []
# for i in range(len(df)):
#     features.append(extract_features(df.iloc[i]["url"]))
# # Concatenate the features and convert to a numpy array
# features = torch.cat(features).numpy()

In [ ]:
!pip install tqdm

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, TensorDataset
import concurrent.futures

# Pastikan menggunakan GPU jika tersedia
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Muat model dan tokenizer yang lebih ringan
from transformers import BertModel, BertTokenizer
import torch

# Load the pre-trained BERT model and tokenizer
# model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

# model_name = "distilbert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name).to(device)
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name).to(device)

def process_batch(batch_indices, texts):
    batch = [texts[i] for i in batch_indices[0]]
    inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=64)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    last_hidden_states = outputs.last_hidden_state
    return torch.mean(last_hidden_states, dim=1).cpu().numpy()

def extract_features_highly_optimized(texts, batch_size=128):
    dataset = TensorDataset(torch.arange(len(texts)))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, prefetch_factor=2)

    features = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        future_to_batch = {executor.submit(process_batch, batch_indices, texts): batch_indices
                           for batch_indices in dataloader}

        for future in tqdm(concurrent.futures.as_completed(future_to_batch), total=len(future_to_batch), desc="Extracting features"):
            features.extend(future.result())

    return np.array(features)

# Fungsi untuk memproses dataset dalam chunk
def process_in_chunks(texts, chunk_size=10000):
    all_features = []
    for i in range(0, len(texts), chunk_size):
        chunk = texts[i:i+chunk_size]
        features = extract_features_highly_optimized(chunk)
        all_features.append(features)
        print(f"Processed chunk {i//chunk_size + 1}/{(len(texts) + chunk_size - 1)//chunk_size}")
    return np.concatenate(all_features)

# Ekstrak fitur dengan metode yang sangat dioptimalkan
features = process_in_chunks(df["url"].tolist())

In [ ]:
features

# Data Processing
---

In [ ]:
types = df['type'].values
types

In [ ]:
print(features.shape)
print(types.shape)

In [ ]:
print(f"Shape of features: {features.shape}")
print(f"Total elements: {features.size}")

# Hitung jumlah fitur per sampel
features_per_sample = features.shape[1] if len(features.shape) > 1 else 1

# Hitung jumlah sampel
num_samples = features.shape[0]

print(f"Number of samples: {num_samples}")
print(f"Features per sample: {features_per_sample}")

# Reshape sesuai dengan jumlah sampel yang sebenarnya
features_reshaped = features.reshape((num_samples, -1))

print(f"Shape after reshaping: {features_reshaped.shape}")

In [ ]:
# # features is a 2D numpy array of size 4000x768
# # labels is a 1D numpy array of size 1000
# # reshape the feature array to size 1000x(768*4)

# features_reshaped = features.reshape((1000, -1))

# # concatenate the feature array with the label array horizontally
# dataset = np.hstack((features_reshaped, types.reshape((-1, 1))))

# # dataset is a 2D numpy array of size 1000x(4000*768+1)
# Assuming you want to work with the first 1000 samples
features_reshaped = features[:1000].reshape((1000, -1))  # Reshape to (1000, 768*4) if needed
types_reshaped = types[:1000].reshape((-1, 1))  # Reshape the label array to (1000, 1)

# Concatenate the feature array with the label array horizontally
dataset = np.hstack((features_reshaped, types_reshaped))

# Now 'dataset' should have the shape (1000, 768*4 + 1)
print(dataset.shape)


In [ ]:
features_reshaped.shape

In [ ]:
dataset

In [ ]:
dataset.shape

# Data Splitting
---

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Convert the training and testing sets back into separate feature and label arrays
X_train, y_train = train_data[:, :-1], train_data[:, -1]
X_test, y_test = test_data[:, :-1], test_data[:, -1]

# Balancing Classes using SMOTE
---

In [ ]:
from imblearn.over_sampling import SMOTE

# Instantiate SMOTE
sm = SMOTE(random_state=42)

# Fit SMOTE to training data
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

y_train_s = pd.Series(y_train)
y_train_res = pd.Series (y_train_res)

# Print class distribution of original and resampled data
print('Class distribution before resampling:', y_train_s.value_counts())
print('Class distribution after resampling:', y_train_res.value_counts())

# Logistic Regression Model
---

In [ ]:
from sklearn.linear_model import LogisticRegression

# Train a logistic regression classifier on the training set
clf = LogisticRegression()
clf.fit(X_train, y_train)

In [ ]:
# Evaluate the classifier on the testing set
score = clf.score(X_test, y_test)
print("Accuracy:", score)

<!-- #### Accuracy - 96% -->

# Model Performance
---

In [ ]:
# Predict the labels of the testing set
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Hitung Precision, Recall, F1-Score dengan average='macro' atau 'weighted'
precision = precision_score(y_test, y_pred, average='macro')  # atau average='weighted'
recall = recall_score(y_test, y_pred, average='macro')        # atau average='weighted'
f1 = f1_score(y_test, y_pred, average='macro')                # atau average='weighted'

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


In [ ]:
# Generate the confusion matrix and classification report
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

In [ ]:
print("Confusion Matrix:\n", cm)

In [ ]:
print("\nClassification Report:\n", cr)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import label_binarize

# Binarize the output for multi-class ROC AUC
y_test_binarized = label_binarize(y_test, classes=[0, 1, 2])  # Update classes based on your problem
n_classes = y_test_binarized.shape[1]

# Get predicted probabilities for each class
y_prob = clf.predict_proba(X_test)

# Compute AUC for each class
auc = {}
for i in range(n_classes):
    auc[i] = roc_auc_score(y_test_binarized[:, i], y_prob[:, i])  # y_prob should be shaped correctly

print("AUC-ROC for each class:", auc)


# Terima Kasih
---